In [1]:
import sys
sys.path.append("../")
from life_cycle.cycle_game import *
from tiles.tiles_moves import *
from tiles.tiles_acces import *
from ui.play_display import *
from life_cycle.play import *
from random import *

def get_nb_empty_rooms(plateau):
    nb_cases_libres = 0
    i = 0
    while i < len(plateau["tiles"]):
        if plateau["tiles"][i] == 0:
            nb_cases_libres += 1
        i += 1
    plateau['nb_cases_libres'] = nb_cases_libres
    return nb_cases_libres

def get_next_alea_tiles(plateau,mode):
    """Retourne une ou deux tuiles(s) dont la disposition (ligne,colonne) est tirée 
       aléatoirement et correspond à un emplacement libre du plateau"""
      
    if mode == "init":
        tuile1 = randint(0,len(plateau["tiles"])-1)
        tuile2 = randint(0,len(plateau["tiles"])-1)
        while tuile2 == tuile1:
            tuile2 = randint(0,len(plateau["tiles"])-1)
            
        return {"mode":"init", 0:{"val":1,"ligne":tuile1//plateau['n'],'colonne':tuile1%plateau['n']},
                1:{"val":2,"ligne":tuile2//plateau['n'],'colonne':tuile2%plateau['n']},'check':True}
    else:
        if is_game_over(plateau):
            return {"mode":"encours","check":False}
        else:
            tuile1 = randint(0,len(plateau["tiles"])-1)
            while plateau['tiles'][tuile1] != 0:
                tuile1 = randint(0,len(plateau["tiles"])-1)
            return {"mode":"encours", 0:{"val":randint(1,3),"ligne":tuile1//plateau['n'],'colonne':tuile1%plateau['n']},
                   'check':True}

def put_next_tiles(plateau,tiles):
    """Permet de placer une ou deux tuiles dans les plateau"""
    if tiles["mode"] == 'init':
        val1 = tiles[0]
        val2 = tiles[1]
        plateau["tiles"][val1['ligne']*plateau['n']+val1["colonne"]] = val1["val"]
        plateau["tiles"][val2['ligne']*plateau['n']+val2["colonne"]] = val2["val"]
    else:
        val1 = tiles[0]
        plateau["tiles"][val1['ligne']*plateau['n']+val1["colonne"]] = val1["val"]

def line_pack(plateau,num_ligne,debut,sens):
    """Tasse les tuiles d'une ligne dans un sens donné
       plateau : dictionnaire contenant le plateau du jeu
       num_ligne : indice de la ligne à "tasser"
       debut : indice à partir duquel se fait le "tassement"
       sens : du "tassement, 1 vers la gauche, 0 vers la droite
    """
    
    assert debut < plateau["n"] and debut >= 0
    assert num_ligne < plateau["n"] and num_ligne >= 0
    
    if sens == 0:
        
        while debut > 0:
            plateau["tiles"][ num_ligne * plateau["n"] + debut ] = plateau["tiles"][ num_ligne * plateau["n"] + debut - 1 ]
            debut -= 1
        plateau["tiles"][ num_ligne * plateau["n"] + debut ] = 0
        
    elif sens == 1:
        
        while debut < plateau["n"] - 1:
            plateau["tiles"][ num_ligne * plateau["n"] + debut ] = plateau["tiles"][ num_ligne * plateau["n"] + debut + 1 ]
            debut += 1
        plateau["tiles"][ num_ligne * plateau["n"] + debut ] = 0
        
            
def colum_pack(plateau, num_col, debut, sens):
    """Tasse les tuiles d'une colonne dans un sens donné
       plateau : dictionnaire contenant le plateau du jeu
       num_col : indice de la colonne à "tasser"
       debut : indice à partir duquel se fait le "tassement"
       sens : du "tassement, 1 vers le haut, 0 vers le bas
    """
    
    assert debut < plateau["n"] and debut >= 0
    assert num_col < plateau["n"] and num_col >= 0
    
    if sens == 0:
        
        while debut > 0:
            plateau["tiles"][ debut * plateau["n"] + num_col ] = plateau["tiles"][ (debut - 1) * plateau["n"] + num_col ]
            debut -= 1
        plateau["tiles"][ debut * plateau["n"] + num_col ] = 0
        
    elif sens == 1:
        
        while debut < plateau["n"] - 1:
            plateau["tiles"][ debut * plateau["n"] + num_col ] = plateau["tiles"][ (debut + 1) * plateau["n"] + num_col ]
            debut += 1
        plateau["tiles"][ debut * plateau["n"] + num_col ] = 0
            
def line_move(plateau, num_lig, sens):
    """ Déplacement des tuiles d'une ligne donnée dans un sens donné 
        en appliquant les règles du jeu Threes
        plateau : dictionnaire contennant le plateau du jeu
        num_lig : indice de la ligne pour laquelle il faut déplacer
                  les tuiles
        sens : sens du déplacement des tuiles, 1 vers la gauche, 0 vers la droite
    """
    assert sens == 1 or sens == 0
    
    p_longueur = plateau["n"] - 1
    
    if sens == 0 :
        
        num_col = p_longueur
        if is_room_empty(plateau, num_lig, num_col) == False:
            while num_col > 0 and is_room_empty(plateau, num_lig, num_col - 1) == False :
                tile_1 = get_value(plateau, num_lig, num_col)
                tile_2 = get_value(plateau, num_lig, num_col - 1)
                if tile_1 == 1 and tile_2 == 2 or tile_1 == 2 and tile_2 == 1:
                    line_pack(plateau, num_lig, num_col, 0)
                    set_value(plateau, num_lig, num_col, 3)
                elif tile_1 == tile_2 and tile_1 != 1 and tile_1 != 2 :
                    line_pack(plateau, num_lig, num_col, 0)
                    set_value(plateau, num_lig, num_col, tile_1 * 2)
                num_col -= 1
                
        num_col = 0
        while num_col <= p_longueur and is_room_empty(plateau, num_lig, num_col):
            num_col += 1
        while num_col <= p_longueur:
            while is_room_empty(plateau, num_lig, num_col):
                line_pack(plateau, num_lig, num_col, 0)
            num_col += 1
            
    else:
        
        num_col = 0 
        if is_room_empty(plateau, num_lig, num_col) == False:
            while num_col < p_longueur and is_room_empty(plateau, num_lig, num_col + 1) == False :
                tile_1 = get_value(plateau, num_lig, num_col)
                tile_2 = get_value(plateau, num_lig, num_col + 1)
                if tile_1 == 1 and tile_2 == 2 or tile_1 == 2 and tile_2 == 1:
                    line_pack(plateau, num_lig, num_col, 1)
                    set_value(plateau, num_lig, num_col, 3)
                elif tile_1 == tile_2 and tile_1 != 1 and tile_1 != 2:
                    line_pack(plateau, num_lig, num_col, 1)
                    set_value(plateau, num_lig, num_col, tile_1 * 2)
                num_col += 1
        
        num_col = p_longueur
        while num_col >= 0 and is_room_empty(plateau, num_lig, num_col):
            num_col -= 1        
        while num_col >= 0:
            while is_room_empty(plateau, num_lig, num_col):
                line_pack(plateau, num_lig, num_col, 1)
            num_col -= 1

def column_move(plateau, num_col, sens):
    """ Déplacement des tuiles d'une colonne donnée dans un sens donné
        en appliquant les règles du jeu Threes
        plateau : dictionnaire contenant le plateau du jeu
        num_col : indice de la colonne pour laquelle il faut déplacer les tuiles
        sens : sens du déplacement des tuiles, 1 vers le haut, 0 vers le bas
        
    """
    assert sens == 1 or sens == 0
    
    p_largeur = plateau["n"] - 1
    
    if sens == 0 :
        
        num_lig = p_largeur
        if is_room_empty(plateau, num_lig, num_col) == False:
            while num_lig > 0 and is_room_empty(plateau, num_lig - 1, num_col) == False :
                tile_1 = get_value(plateau, num_lig, num_col)
                tile_2 = get_value(plateau, num_lig - 1, num_col)
                if tile_1 == 1 and tile_2 == 2 or tile_1 == 2 and tile_2 == 1:
                    colum_pack(plateau, num_col, num_lig, 0)
                    set_value(plateau, num_lig, num_col, 3)
                elif tile_1 == tile_2 and tile_1 != 1 and tile_1 != 2 :
                    colum_pack(plateau, num_col, num_lig, 0)
                    set_value(plateau, num_lig, num_col, tile_1 * 2)
                num_lig -= 1
                
        num_lig = 0
        while num_lig <= p_largeur and  is_room_empty(plateau, num_lig, num_col):
            num_lig += 1
        while num_lig <= p_largeur:
            while is_room_empty(plateau, num_lig, num_col):
                colum_pack(plateau, num_col, num_lig, 0)
            num_lig += 1
            
    else:
        
        num_lig = 0 
        if is_room_empty(plateau, num_lig, num_col) == False:
            while num_lig < p_largeur and is_room_empty(plateau, num_lig + 1, num_col) == False :
                tile_1 = get_value(plateau, num_lig, num_col)
                tile_2 = get_value(plateau, num_lig + 1, num_col)
                if tile_1 == 1 and tile_2 == 2 or tile_1 == 2 and tile_2 == 1:
                    colum_pack(plateau, num_col, num_lig, 1)
                    set_value(plateau, num_lig, num_col, 3)
                elif tile_1 == tile_2 and tile_1 != 1 and tile_1 != 2:
                    colum_pack(plateau, num_col, num_lig, 1)
                    set_value(plateau, num_lig, num_col, tile_1 * 2)
                num_lig += 1
        
        num_lig = p_largeur
        while num_lig >= 0 and is_room_empty(plateau, num_lig, num_col):
            num_lig -= 1        
        while num_lig >= 0:
            while is_room_empty(plateau, num_lig, num_col):
                colum_pack(plateau, num_col, num_lig, 1)
            num_lig -= 1
            
def lines_move(plateau, sens):
    """ Déplace les tuiles de toutes les lignes du plateau
        dans un sens donné en appliquant les règles du jeu Threes
        plateau : dictionnaire contenant le plateau du jeu
        sens : sens du déplacement; 1 vers la gauche, 0 vers la droite
    """
    i = 0
    while i < plateau["n"]:
        line_move(plateau, i, sens)
        i += 1
        
def columns_move(plateau, sens):
    """ Déplace les tuiles de toutes les colonnes du plateau
        dans un sens donné en appliquant les règles du jeu Threes
        plateau : dictionnaire contenant le plateau du jeu
        sens : sens du déplacement; 1 vers le haut, 0 vers le bas
    """
    i = 0
    while i < plateau["n"]:
        column_move(plateau, i, sens)
        i += 1
        
def play_move(plateau, sens):
    """ Déplace les tuiles du plateau dans un sens donné en
        appliquant les règles du jeu Threes
        plateau : plateau de jeu
        sens : sens de déplacement des tuiles
    """
    if sens == "b":
        columns_move(plateau,0)
    elif sens == "h":
        columns_move(plateau,1)
    elif sens == "d":
        lines_move(plateau, 0)
    elif sens == "g":
        lines_move(plateau, 1)
    else:
        print("'sens' ne contient pas une bonne valeur.")
    


In [6]:
def test_get_nb_empty_rooms():
    p = {'n':4,'nb_cases_libres':10,'tiles':[6,2,3,2,0,2,6,2,0,2,2,0,1,0,0,0]}
    get_nb_empty_rooms(p)
    assert p['nb_cases_libres'] == 6
    print(' Test get_nb_empty_rooms : ok')
    
test_get_nb_empty_rooms()

 Test get_nb_empty_rooms : ok


In [1]:
p = p = {'n':4,'nb_cases_libres':6,'tiles':[6,2,3,2,0,2,6,2,1,2,2,1,1,1,0,1]}
simple_display(p)

NameError: name 'simple_display' is not defined